In [2]:
from dataclasses import dataclass
from functools import cached_property
from pathlib import Path
import sqlite3
import yaml
conn = sqlite3.connect('mah_dope_meads.db')

In [3]:
@dataclass
class MeadTracker:

    db_name: str = "mah_dope_meads.db"

    def __post_init__(self) -> None:
        """
        creates conn object
        """
        self.conn: sqlite3.Connection = sqlite3.connect(self.db_name)
        self.run_init_cmds()

    @cached_property
    def cfg(self) -> dict:
        """
        captures cfg yaml
        """
        return yaml.safe_load(Path("cfg.yml").read_text())

    @property
    def init_cmds(self) -> None:
        """
        returns all ddl_cmds from cfg
        """
        # capture all init_statements if there is actual text (not just placedholders)
        cmds: dict = self.cfg.get("sql", {}).get('init_statements',{})
        valid: dict = [c for c in cmds.values() if c]
        return valid

    def run_sql(self, cmd: str) -> None:
        """
        run and commits a command
        """
        with self.conn:
            self.conn.execute(cmd)

    def run_init_cmds(self) -> None:
        """
        runs all ddl commands at startup
        because they all have `if not exists` we're golden 🤩
        """
        for cmd in self.init_cmds:
            print(cmd)
            self.run_sql(cmd)

In [4]:
md = MeadTracker()
md.run_init_cmds()


create table if not exists meads (
  id integer primary key
  , mead_name string not null
  , start_date date not null
  , yeast_used string not null
  , sugar_source string not null
  , starting_gravity float
  , potential_abv float
  , yield_in_oz float
  , unique(Mead_name, start_date)
);

create table if not exists mead_notes (
  id integer primary key
  , mead_id integer not null
  , note string not null
  , note_date date not null
  , foreign key(mead_id) references meads(id)
);

create table if not exists activity (
  id integer primary key
  , mead_id integer not null
  , act_name string not null
  , act_date date not null
  , check (act_name in ('started','racked','bottled','modified','other'))
  , foreign key(mead_id) references meads(id)
);

create table if not exists abv_measurements (
  id integer primary  key
  , mead_id integer not null
  , sample_date date not null
  , curr_gravity float not null
  , curr_abv float not null
  , days_since_start int
  , pct_to_pot_abv fl